# Retrivers

> Retrievers implement the Runnable interface(LCEL): `invoke`, `ainvoke`, `stream`, `astream`, `batch`, `abatch`, `astream_log` <br>
> Retrievers accept a string <font color=blue>query as input</font> and return a list of `Document's` as output.

In [1]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

In [2]:
full_text = open("./input/state_of_the_union.txt", "r").read()

In [4]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
texts = text_splitter.split_text(full_text)

In [6]:
embeddings = OpenAIEmbeddings()

In [7]:
db = FAISS.from_texts(texts, embeddings)

In [8]:
retriever = db.as_retriever()

In [9]:
retrieved_docs = retriever.invoke("What did the president say about Ketanji Brown Jackson?")

In [10]:
print(retrieved_docs[0].page_content)

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence. 

A former top litigator in private practice. A former federal public defender. And from a family of public school educators and police officers. A consensus builder. Since she’s been nominated, she’s received a broad range of support—from the Fraternal Order of Police to former judges appointed by Democrats and Republicans. 

And if we are to advance liberty and justice, we need to secure the Border and fix the immigration system. 

We can do both. At our border, we’ve installed new technology like cutting-edge scanners to better detect drug smuggling.  

We’ve set up joint patrols with Mexico and Guatemala to catch more human traffickers.


In [11]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

In [12]:
template = """Answer the question based only on the following context:

{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [13]:
model = ChatOpenAI()

In [14]:
def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])

In [15]:
chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [16]:
response = chain.invoke("What did the president say about technology?")

In [17]:
type(response)

str

In [18]:
print(response)

The president said that technology is important for competing for the best jobs of the future and that passing the Bipartisan Innovation Act will make record investments in emerging technologies and American manufacturing. Additionally, the president mentioned Intel's plan to build a semiconductor "mega site" and increase their investment from $20 billion to $100 billion, which would be one of the biggest investments in manufacturing in American history.
